In [1]:
from pyspark.sql.functions import *

inputPath = "dbfs:/mnt/yelp-mount"

In [2]:
userDf = spark.read.json("/FileStore/tables/user.json")
reviewDf = spark.read.json(inputPath+"/review.json")
filteredBusinessDf = spark.read.json('/FileStore/tables/filtered_business.json')

In [3]:
display(userDf)

In [4]:
userDf = userDf.withColumn("user_unique_user_id",monotonically_increasing_id())
filteredBusinessDf = filteredBusinessDf.withColumn("unique_business_id",monotonically_increasing_id())

In [5]:
filteredBusinessDf.createOrReplaceTempView("business")
userDf.createOrReplaceTempView("user")
reviewDf.createOrReplaceTempView("review")

In [6]:
display(userDf)

In [7]:
display(filteredBusinessDf)

In [8]:
%sql select * from review inner join business on business.business_id = review.business_id inner join user on user.user_id = review.user_id

In [9]:
df = sqlContext.sql('select user_unique_user_id,unique_business_id, review.stars, state from review inner join business on business.business_id = review.business_id inner join user on user.user_id = review.user_id where user_unique_user_id < 2000000000 and unique_business_id < 2000000000')

In [10]:
%fs ls /FileStore

In [11]:
%fs rm -r /FileStore/AZ_business.json

In [12]:
%fs rm -r /FileStore/NC_business.json

In [13]:
%fs rm -r /FileStore/NV_business.json

In [14]:
%fs rm -r /FileStore/OH_business.json

In [15]:
%fs rm -r /FileStore/ON_business.json

In [16]:
OH_rest_df = df[df['state']=="OH"]
OH_rest_df.write.json("/FileStore/OH_business.json")

NC_rest_df = df[df['state']=="NC"]
NC_rest_df.write.json("/FileStore/NC_business.json")

ON_rest_df = df[df['state']=="ON"]
ON_rest_df.write.json("/FileStore/ON_business.json")

NV_rest_df = df[df['state']=="NV"]
NV_rest_df.write.json("/FileStore/NV_business.json")

AZ_rest_df = df[df['state']=="AZ"]
AZ_rest_df.write.json("/FileStore/AZ_business.json")

In [17]:
df.describe()

In [18]:
display(df)